# **Описание работы:**

Анализ продаж - это процесс изучения данных о продажах товаров с целью получения ценной информации и понимания факторов, влияющих на объемы продаж. Он включает в себя исследование количественных и качественных данных, визуализацию результатов и выявление ключевых трендов и паттернов. Анализ продаж помогает компаниям принимать обоснованные бизнес-решения, оптимизировать стратегии маркетинга, управлять ассортиментом товаров, определить приоритеты в управлении запасами и повысить общую эффективность бизнеса. Это позволяет компаниям быть более конкурентоспособными и адаптивными к изменяющимся требованиям рынка.

В ходе анализа продаж ответим на ряд вопросов, таких как:

* Каковы общие тенденции продаж товаров за определенный период времени?
* Какие товары являются наиболее и наименее популярными среди клиентов?
* Какие факторы, такие как цена, сезонность и другие, оказывают наибольшее влияние на объемы продаж?
* Какие товары являются наиболее прибыльными для компании?

### **Основные этапы работы:**

* Загрузка и изучение структуры данных и их предварительная обработка.
* Исследовательский анализ данных (EDA): анализ набора данных, включая визуализации и статистические сводки, необходимые для получения представления о распределении и взаимосвязях между переменными.
* Интерпретация результатов исследования и подведение итогов: извлечение полезной информации из исследования и составление кратких, содержательных выводов.

### **Описание набора данных**:

* Order ID - Идентификатор заказа. Каждый заказ получает свой уникальный идентификатор.
* Product - Продукт (товар), который был продан.
* Quantity Ordered - Заказанное количество товара.
* Price Each - Цена товара. Валюта - Доллар США (знак валюты - $)
* Order Date - Дата заказа товара.
* Purchase Address - Адрес доставки товара.

# Загрузка и изучение структуры данных и их предварительная обработка

In [ ]:
import numpy as np                  # Импорт библиотеки NumPy
import pandas as pd                 # Импорт библиотеки Pandas
import seaborn as sns               # Импорт библиотеки SeaBorn
import matplotlib.pyplot as plt     # Импорт библиотеки MatPlotLib
import os                           # Импорт модуля os
import phik                         # Импорт модуля phik
%matplotlib inline                
# Строка для отображения графиков в текущем ноутбуке.

plt.style.use('bmh')                # Устанавлием стиль графиков Bayesian Methods for Hackers по умолчанию

In [ ]:
files = [file for file in os.listdir('../input/sales-product-data')]    # Создание списка, содержащий имена файлов в указанной директории

all_months_data = pd.DataFrame()                                        # Создание пустого датафрейма

for file in files:                                                      # Создание цикла
    data = pd.read_csv("../input/sales-product-data/" + file)           # Считывания содержимого CSV-файла
    all_months_data = pd.concat([all_months_data, data])                # Объединение содержимого датафреймов    
    
all_months_data.to_csv("all_data.csv", index=False)                     # Запись датафрейма в CSV-файл

df = pd.read_csv('all_data.csv')                                        # Чтение CSV-файла
df.head(5)                                                              # Отображение нескольких строк датафрейма

In [ ]:
df.info()    # Просмотр структуры датафрейма

In [ ]:
df.isna().mean()    # Просмотр доли пустых значений

In [ ]:
df.describe()    # Просмотр статистических данных

In [ ]:
df.query('`Order Date` == "Order Date"')    # Проверка на наличие ложных данных (выбросов)

In [ ]:
df = df.dropna(how = 'all')                   # Удаление строк с пустыми значениями
df = df[df['Order Date'].str[0:2] != 'Or']    # Удаление ложных данных

In [ ]:
display(df.isna().mean())    # Проверка структуры датафрейма
df.describe() 

In [ ]:
df['Quantity Ordered'], df['Price Each'] = df['Quantity Ordered'].astype('int64'), df['Price Each'].astype('float')    # Преобразование типа данных в столбцах
df['Order Date'] = pd.to_datetime(df['Order Date'])    # Преобразование строкового типа данных в тип данных 'datetime' (дата и время)

In [ ]:
df['Year'] = df['Order Date'].dt.year                       # Создание столбца только со значением года

df['Month_Number'] = df['Order Date'].dt.month              # Создание столбца только с числовым значением номера месяца

df['Month'] = df['Order Date'].dt.month_name()              # Создание столбца только с названием месяца

df['Day_Number'] = df['Order Date'].dt.dayofweek            # Создание столбца только с числовым значением номера дня недели

df['Day'] = df['Order Date'].dt.day_name()                  # Создание столбца только с названием дня недели

df['Hour'] = df['Order Date'].dt.hour                       # Создание столбца только со значением часа
    
df['Sales'] = df['Quantity Ordered'] * df['Price Each']     # Создание столбца со значением итоговой выручки
    
df['Cities'] = df['Purchase Address'].apply(lambda x: x.split(',')[1])                 # Создание столбца с названиями городов

df['State'] = df['Purchase Address'].apply(lambda x: x.split(',')[2].split(' ')[1])    # Создание столбца с названиями штатов

df = df.drop(['Purchase Address'], axis = 1)                # Удаление более не нужного нам столбца из датафрейма

In [ ]:
df.groupby('Year')['Order ID'].agg('count')    # Группировка по годам и подсчет количества значений 'Order ID'

In [ ]:
df = df[df['Order Date'].dt.year != 2020]    # Перезаписываем датафрейм, исключая все года, кроме 2019, чтобы исключить дисбаланс данных.

In [ ]:
df.groupby(['Cities','State']).agg('count').reset_index()[['Cities','State']]    # Группировка по городам и штатам

Как видно выше, в двух разных штатах существуют города с одинаковыми названиями, исправим это.

In [ ]:
df.loc[df['State'] == 'ME', 'Cities'] = 'Portland (ME)'    # Изменение названия города с учетом штата, в котором он находится
df.loc[df['State'] == 'OR', 'Cities'] = 'Portland (OR)'

# Исследовательский анализ данных

In [ ]:
df.describe()    # Просмотр структуры данных измененного датафрейма

In [ ]:
df_matrix = df.drop(['Cities','State','Product','Order ID','Month','Year','Day','Order Date'], axis = 1).phik_matrix()
# Удаление всех неподходящих значений из датафрейма для последующего вычисления матрицы коэффициентов корреляции для заданного набора данных.

cmap = sns.cubehelix_palette(start=1, rot=-1, dark=0.1, light=0.5, as_cmap=True)    # Создание цветовой палитры
sns.heatmap(df_matrix, annot = True, fmt='.3f', cmap = cmap)                        # Визуализация матрицы корреляции
plt.xticks(rotation = 45, ha = 'right')                                             # Устанавливаем поворот делений оси X
plt.show()                                                                          # Отображение матрицы

In [ ]:
def proportion_table(Column: str, rd: int)-> pd.DataFrame:

    """
    Создание функции, которая на входе принимает следующие данные:
     - Column - Параметр, который принимает столбец данных для анализа пропорций
     - rd - Параметр, который определяет количество десятичных знаков после округления
    """
    df_gr_count = df.groupby(Column)['Order ID'].agg(['count']).reset_index()                         # Группировка и подсчет значений      
    df_count = df['Order ID'].agg(['count']).reset_index()                                            # Подсчет количества всех заказов
    df_gr_count['proportion'] = (df_gr_count['count'] / df_count['Order ID'].values*100).round(rd)    # Создание столбца со значением доли и его округление
    df_gr_count = df_gr_count.sort_values(by = 'proportion', ascending = False)                       # Сортировка по убыванию
    return display(df_gr_count)                                                                       # Возвращение искомых значений

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))         # Задаем расположение и размеры графиков на графической сетке

sns.histplot(x = df['Quantity Ordered'], ax = axes[0])              # Создание гистограммы количества купленного товара
axes[0].set_xlabel('Заказанное количество товара')                  # Задаем название оси X
axes[0].set_ylabel('Количество заказов')                            # Задаем название оси Y

sns.kdeplot(df['Quantity Ordered'], ax = axes[1], color = 'red')    # Создание графика плотности распределения (PDF)
axes[1].set_xlabel('Заказанное количество товара')
axes[1].set_ylabel('Плотность')

plt.show()

proportion_table('Quantity Ordered', 3)                             # Вызов функции

В итоге видим, 90,6% товаров куплены в единичном экземпляре. Далее идет резкое уменьшение доли заказанного количества товаров. Так заказы сразу двух товаров составлют около 7%, а трех - всего 1,5%. В целом, основываясь на предоставленных данных, можно сказать, что большинство клиентов предпочитают делать единичные покупки или заказывать небольшие партии товаров, в то время как заказы на большое количество товаров являются крайне редким явлением.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))  

median = df['Price Each'].median()                                    # Находим среднее значения цены товаров
mean = df['Price Each'].mean()                                        # Находим медиану цены товаров  

df['Price Each'].hist(bins = 25, ax = axes[0])                        # Создание гистограммы цены товаров
axes[0].set_xlabel('Цена товара')
axes[0].set_ylabel('Количество заказов')  

sns.kdeplot(df['Price Each'], ax = axes[1], color = 'red')
axes[1].set_xlabel('Цена товара')
axes[1].set_ylabel('Плотность')  
axes[1].axvline(mean, color='g', linestyle='--', label='Mean')        # Определение параметров линии среднего значения на графике
axes[1].axvline(median, color='b', linestyle='--', label='Median')    # Определение параметров линии медианы на графике
axes[1].legend()                                                      # Отображение легенды на графике

plt.show()

proportion_table('Price Each', 2) 

Согласно данным выше, видим что около 55% заказанных товаров имели цену от 2,99 до 14,95. В целом, представленные данные позволяют увидеть распределение цен на товары и их соотношение в общем объеме продаж.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))                                       

df_sales = df.groupby('Product')['Quantity Ordered'].agg('sum').sort_values(ascending = False)    # Группировка и подсчет количества заказанных продуктов и сортировка по убыванию
df_prices = df.groupby('Product')['Price Each'].mean().sort_values()                              # Группировка по продуктам и сортировка по возрастанию цены

axes[0].bar(df_sales.index, df_sales.values)                                                      # Построение диаграммы количества заказанных продуктов
axes[0].set_xticks(range(len(df_sales.index)))
axes[0].set_xticklabels(df_sales.index, rotation=45, ha='right')
axes[0].set_xlabel('Товары')
axes[0].set_ylabel('Количество заказанных товаров')
                                                                                
axes[1].plot(df_prices.index, df_prices.values, color='red')
axes[1].set_xticks(range(len(df_sales.index)))
axes[1].set_xticklabels(df_prices.index, rotation=45, ha='right')                                 # Построение графика цены товара
axes[1].set_ylabel('Цена товара ($)')
axes[1].set_xlabel('Товары')

plt.show()

Как видно на графике выше, чем ниже цена товара, тем чаще и в большем количестве его покупают. Так, в пятерку самых продаваемых товаров входят батарейки, зарядные устройства и наушники в связи с их относительно небольшой стоимостью. Наименее продаваемые товары, такие как стиральные машины, смартфоны и ноутбуки, имеют значительно большую стоимость соответственно.

In [ ]:
plt.figure(figsize=(16, 6))

df_sales = df.groupby('Product')['Sales'].agg('sum').sort_values(ascending = False)    # Группировка и подсчет итоговой выручки

plt.bar(df_sales.index, df_sales.values)                                               # Создание диаграммы итоговой выручки
plt.xticks(rotation=45, ha = 'right')
plt.ticklabel_format(style='plain', axis='y')                                          # Редактирование формата значений оси Y
plt.xlabel('Товары')
plt.ylabel('Итоговая выручка ($)')
plt.show()

Согласно графику, наиболее прибыльные товары - смартфоны и ноутбуки, а наименее прибыльные - батарейки, зарядные устройства и наушники. Такие значительные отличия объясняются тем, что разница в цене между этими товарами в несколько десятков раз, а количество покупок - всего в несколько раз, что было выяснено ранее.

In [ ]:
df.groupby('Year')['Sales'].agg(sum)    # Подсчет суммарной выручки за год

In [ ]:
# Подсчет доли по количеству заказанных товаров и по итоговой выручки для каждого продукта(товара)
df_p = df.groupby('Product')['Sales'].agg(['count','sum']).reset_index()
df_pc = df['Sales'].agg(['sum']).reset_index()
df_pcf = df['Sales'].agg(['count']).reset_index()
df_p['count_proportion'] = ((df_p['count'] / df_pcf['Sales'].values)*100).round(2)    
df_p['sum_proportion'] = ((df_p['sum'] / df_pc['Sales'].values)*100).round(2)
df_f = df_p.sort_values(by = 'sum_proportion', ascending = False)
df_f

Предоставленные данные позволяют выделить три группы товаров относительно их доле продаж и итоговой выручки. Начнем с наиболее значимой группы.

In [ ]:
display(df_f.head(7))    # Отображение первой группы

В этой таблице представлены наиболее прибыльные товары, их суммарная выручка состовляет 79,5% от общей итоговой выручки, а доля - 26,5% от общей доли продаж.

In [ ]:
display(df_f.iloc[7:12])    # Отображение второй группы

Здесь же представлены менее прибыльные товары, которые все еще приносят значимую выручку, которая составляет 15% от общей. Доля продаж около 17,5%.

In [ ]:
display(df_f.tail(7))    # Отображение третьей группы

В этой таблице можно увидеть наименее прибыльные товары, которые имеют наименьшию значимость для бизнеса. При доли продаж 56% - итоговая выручка составляет всего около 5,5% от общей итоговой выручки.

In [ ]:
from itertools import combinations    # Импорт функции combinations из модуля itertools
from collections import Counter       # Импорт класса Counter из модуля collections

df_dupl = df[df['Order ID'].duplicated(keep=False)]             # Находим дубликаты среди 'Order ID'
df_dupl = df_dupl[['Order ID','Product']]                       # Сохраняем в датафрейме только нужные столбцы

df_dupl['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))    # Создание столбца со названиями продуктов и группировкой по столбцу 'Order ID' 

df_dupl = df_dupl[['Order ID', 'Grouped']].drop_duplicates()    # Удаление дубликатов по столбцам 'Order ID' и 'Grouped'

count = Counter()                                               # Обьявление переменной
for row in df_dupl['Grouped']:                                  # Создание цикла для нахождения всех комбинаций в столбце 'Grouped' и подсчет их количества
    row_list = row.split(',')                           
    count.update(Counter(combinations(row_list, 2)))    
for key, value in count.most_common(5):                         # Создание цикла для отображения всех комбинаций 
    print(key, value)
    key_comb=list(key)
    key_comb.reverse()
    key_comb = tuple(key_comb)
    print(key_comb, count.get(key_comb))

В данном списке представлены комбинации товаров и количество заказов, в которых их купили. Чаще всего вместе со смартфонами заказывают зарядные устройства и наушники. Эти сведения могут помочь в настройке рекламных объявлений при покупке того или иного товара для увеличения среднего чека платящего пользователя.

In [ ]:
def barplot_date_func(Column_Sort: str, Column: str, size_w: int, size_l: int):

    """
    Создание функции, которая на входе принимает следующие данные:
     - Column_Sort - Параметр, который принимает столбец данных для сортировки
     - Column - Параметр, который принимает столбец данных для построение диаграммы
     - size_w - Параметр, который определяет ширину графика
     - size_l - Параметр, который определяет длину графика
    """
    df_sort_sales = df.sort_values(by = Column_Sort).groupby([Column_Sort,Column])['Sales'].agg('sum').reset_index()

    plt.figure(figsize=(size_w, size_l))
    sns.barplot(x=df_sort_sales[Column], y=df_sort_sales['Sales'])    # Создание диаграммы продаж
    plt.xlabel(Column)
    plt.ylabel('Sales ($)')
    plt.ticklabel_format(style='plain', axis='y')
    plt.show()
    return display(df_sort_sales)                                               

In [ ]:
barplot_date_func('Month_Number', 'Month', 12, 6)

Исходя из данных выше, видно, что итоговая выручка имеет ярко выраженную сезонность. Декабрь - наиболее прибыльный месяц, причин этому может быть несколько:
* Наличие праздников, таких как Рождество и Новый Год
* Скидки и распродажи
* Бонусы и ежегодные премии

In [ ]:
barplot_date_func('Day_Number', 'Day', 8, 6)

На протяжении недели общий объем продаж остается относительно стабильным (равномерное распределение). Разница между наивысшим и наименьшим значением выручки незначительна. Нет явного тренда в изменении итоговой выручки в течение недели.

In [ ]:
df_hour_sort_sales = df.sort_values(by = 'Hour').groupby(['Hour'])['Sales'].agg('sum').reset_index()   # Создание диаграммы продаж по часам
plt.figure(figsize=(16, 6))
sns.barplot(x=df_hour_sort_sales['Hour'], y=df_hour_sort_sales['Sales'])
plt.xlabel('Hour')
plt.ylabel('Sales ($)')
plt.ticklabel_format(style='plain', axis='y')
plt.show()

Основываясь на предоставленных данных, можно утверждать следующее:
* Заметен значительный рост выручки утром и в дневное время (с 6:00 до 13:00). В это время покупатели, возможно, делают покупки перед рабочим днем или в пути на работу.
* Вечерние часы (с 18:00 до 22:00) имеют наибольшие показатели выручки. Это может быть связано с тем, что люди совершают больше покупок в свободное от работы время, что способствует росту спроса.

In [ ]:
def barplot_loc_func(Column: str, size_w: int, size_l: int, rot: int, ha: str):

    """
    Создание функции, которая на входе принимает следующие данные:
     - Column - Параметр, который принимает столбец данных для построение диаграммы
     - size_w - Параметр, который определяет ширину графика
     - size_l - Параметр, который определяет длину графика
     - rot - Параметр, который принимает угол поворота делений по оси
     - ha - Параметр, выравнивания значений по горизонтали
    """
    df_sales = df.groupby([Column])['Sales'].agg('sum').reset_index()
    df_sales = df_sales.sort_values(by = 'Sales', ascending = False)
    plt.figure(figsize=(size_w, size_l))
    sns.barplot(x=df_sales[Column], y=df_sales['Sales'])    # Создание диаграммы по продажам
    plt.xlabel(Column)
    plt.ylabel('Sales ($)')
    plt.xticks(rotation=rot, ha = ha)
    plt.ticklabel_format(style='plain', axis='y')
    plt.show() 
    return display(df_sales)                                    

In [ ]:
barplot_loc_func('Cities', 8, 6, 45, 'right')

San Francisco значительно превосходит другие города в списке по обьему продаж. Этот город генерирует наибольший итоговую выручку, достигая значения 8 259 719 $.

In [ ]:
barplot_loc_func('State', 8, 6, 0, 'center')

Калифорния (CA) имеет самый высокий уровень выручки среди всех штатов в списке. В этот в штат входят два самых прибыльных города - San Francisco и Los Angeles.

# Интерпретация результатов исследования и подведение итогов

В ходе исследования установлено следующее:

Покупательские предпочтения:
* Большинство клиентов предпочитает совершать отдельные покупки или заказывать небольшие партии товаров, поскольку 90,6% товаров были приобретены в единичном экземпляре. При заказе двух и более экземпляров наблюдается резкое уменьшение доли заказов. В итоге можно сделать вывод, что заказы на большое количество товаров являются редким явлением.

Распределение цен на товары:
* Около 55% заказанных товаров имели цену от 2,99 до 14,95 доллара. Это указывает на то, что большинство покупателей предпочитает товары с относительно низкой стоимостью.
* Самые продаваемые товары, такие как батарейки, зарядные устройства и наушники, имеют небольшую стоимость. С другой стороны, товары с высокой стоимостью, такие как смартфоны и ноутбуки, продаются значительно реже. Доля продаж стиральных машин 'LG Washing Machine' и 'LG Dryer' составляет менее 0,5%, что является наименьшим значением среди всех товаров.

Прибыльность товаров:
* В ходе исследования были выделены три группы товаров по их влиянию на суммарную выручку. Так, смартфоны и ноутбуки оказываются наиболее прибыльными товарами, в то время как батарейки, зарядные устройства и наушники являются наименее прибыльными. 'Macbook Pro Laptop' принес наибольшую выручку среди всех ноутбуков - 8 035 900 долларов, а среди смартфонов - 'iPhone' - 4 792 900 долларов.

Сезонность выручки:
* Суммарная выручка за год составляет 34 483 365 долларов.
* Декабрь является наиболее прибыльным месяцем - 4 613 443 доллара. В течение всего года происходят заметные изменения в объеме выручки, что может связано со многими событиями, такими как различного рода праздники, технологические тренды, акции и распродажи.
* В течение недели общий объем продаж остается относительно стабильным, без явного тренда изменения выручки.
* Наблюдается значительное изменение объема продаж в течение дня. Утром и в обеденное время (с 6:00 до 13:00) происходит рост выручки. Наибольшие значения выручки отмечаются вечером (с 18:00 до 22:00). Это объясняется тем, что люди имеют больше свободного времени после окончания рабочего дня и проявляют больший интерес к совершению покупок в этот период. Таким образом, время суток оказывает влияние на объем продаж, при этом утренние и вечерние часы характеризуются более высокими значениями выручки. Эти выводы могут быть полезны для планирования маркетинговых активностей и управления запасами товаров.

Анализ местоположения:
* Исходя из проведенного анализа данных, можно сделать вывод, что город San Francisco является лидером с наивысшей общей выручкой в размере 8 259 719 долларов. Кроме того, штат Калифорния (CA), в котором находятся города San Francisco и Los Angeles, обладает самым высоким уровнем выручки среди всех штатов. Эти результаты подчеркивают важность учета местоположения при анализе и планировании маркетинговых стратегий и распределения ресурсов для максимизации прибыли.